<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
<center>Автор материала: программист-исследователь Mail.Ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ [Юрий Кашницкий](https://yorko.github.io/)

# <center> Capstone проект №1 <br> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 5.  Соревнование Kaggle "Catch Me If You Can"

На этой неделе мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали на 4 неделе. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) Kaggle по идентификации пользователей и сделаем в нем первые посылки. По итогам этой недели дополнительные баллы получат те, кто попадет в топ-30 публичного лидерборда соревнования.

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
**Также рекомендуется вернуться и просмотреть [задание](https://www.coursera.org/learn/supervised-learning/programming/t2Idc/linieinaia-rieghriessiia-i-stokhastichieskii-ghradiientnyi-spusk) "Линейная регрессия и стохастический градиентный спуск" 1 недели 2 курса специализации.**

### Задание
1. Заполните код в этой тетрадке 
2. Если вы проходите специализацию Яндеса и МФТИ, пошлите тетрадку в соответствующем Peer Review. <br> Если вы проходите курс ODS, выберите ответы в [веб-форме](https://docs.google.com/forms/d/1pLsegkAICL9PzOLyAeH9DmDOBfktte0l8JW75uWcTng). 

In [3]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier, LogisticRegression, LogisticRegressionCV
from sklearn.metrics import roc_auc_score, accuracy_score
from datetime import datetime

**Считаем данные [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [4]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = ''

In [5]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')

**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [7]:
train_test_df = pd.concat([train_df, test_df])

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   site1   253561 non-null  int64  
 1   time1   253561 non-null  object 
 2   site2   250098 non-null  float64
 3   time2   250098 non-null  object 
 4   site3   246919 non-null  float64
 5   time3   246919 non-null  object 
 6   site4   244321 non-null  float64
 7   time4   244321 non-null  object 
 8   site5   241829 non-null  float64
 9   time5   241829 non-null  object 
 10  site6   239495 non-null  float64
 11  time6   239495 non-null  object 
 12  site7   237297 non-null  float64
 13  time7   237297 non-null  object 
 14  site8   235224 non-null  float64
 15  time8   235224 non-null  object 
 16  site9   233084 non-null  float64
 17  time9   233084 non-null  object 
 18  site10  231052 non-null  float64
 19  time10  231052 non-null  object 
 20  target  253561 non-null  int64  
dtypes: float64

In [9]:
test_df.head(3)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36


In [10]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   site1   82797 non-null  int64  
 1   time1   82797 non-null  object 
 2   site2   81308 non-null  float64
 3   time2   81308 non-null  object 
 4   site3   80075 non-null  float64
 5   time3   80075 non-null  object 
 6   site4   79182 non-null  float64
 7   time4   79182 non-null  object 
 8   site5   78341 non-null  float64
 9   time5   78341 non-null  object 
 10  site6   77566 non-null  float64
 11  time6   77566 non-null  object 
 12  site7   76840 non-null  float64
 13  time7   76840 non-null  object 
 14  site8   76151 non-null  float64
 15  time8   76151 non-null  object 
 16  site9   75484 non-null  float64
 17  time9   75484 non-null  object 
 18  site10  74806 non-null  float64
 19  time10  74806 non-null  object 
dtypes: float64(9), int64(1), object(10)
memory usage: 13.3+ MB


**В обучающей выборке – 2297 сессий одного пользователя (Alice) и 251264 сессий – других пользователей, не Элис. Дисбаланс классов очень сильный, и смотреть на долю верных ответов (accuracy) непоказательно.**

In [11]:
train_df['target'].value_counts()

0    251264
1      2297
Name: target, dtype: int64

**Пока для прогноза будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [12]:
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')

-----------------------------------

In [13]:
def new_df(dataframe):
    sites = ['site' + str(i) for i in range(1, 11)]
    new_df_train = dataframe[[site for site in sites]].fillna(0).astype('int')
    new_df_train['time1'] = dataframe['time1']
    new_df_train['time'] = pd.to_datetime(new_df_train['time1'])
    del new_df_train['time1']
    
    new_df_train['day_of_week'] = np.nan
    new_df_train['start_hour'] = np.nan
    new_df_train['month'] = np.nan
    for i in range(new_df_train.shape[0]):
        new_df_train['day_of_week'].iloc[i] = datetime.isoweekday(new_df_train['time'].iloc[i])
        new_df_train['start_hour'].iloc[i] = new_df_train['time'].iloc[i].hour
        new_df_train['month'].iloc[i] = new_df_train['time'].iloc[i].month
    new_df_train['day_of_week'] = new_df_train['day_of_week'].astype('int')
    new_df_train['start_hour'] = new_df_train['start_hour'].astype('int')
    new_df_train['month'] = new_df_train['month'].astype('int')
    del new_df_train['time']
    
    return new_df_train

In [14]:
def time_df(dataframe):
    time = ['time'+str(i) for i in range(1,11)]
    time_df = dataframe[[t for t in time]]
    for i in range(time_df.shape[1]):
        time_df.iloc[:,i] = pd.to_datetime(time_df.iloc[:,i])
    time_df['delta'] = time_df.max(axis = 1) - time_df.min(axis = 1)
    return time_df

In [15]:
time_train = time_df(train_df)
time_train.head(3)

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10,delta
session_id,,,,,,,,,,,
1,2014-02-20 10:02:45,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,00:00:00
2,2014-02-22 11:19:50,2014-02-22 11:19:50,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:52,2014-02-22 11:19:52,2014-02-22 11:20:15,2014-02-22 11:20:16,00:00:26
3,2013-12-16 16:40:17,2013-12-16 16:40:18,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:20,2013-12-16 16:40:21,2013-12-16 16:40:22,2013-12-16 16:40:24,00:00:07


In [16]:
time_test = time_df(test_df)
time_test.head(3)

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10,delta
session_id,,,,,,,,,,,
1,2014-10-04 11:19:53,2014-10-04 11:19:53,2014-10-04 11:19:54,2014-10-04 11:19:54,2014-10-04 11:19:54,2014-10-04 11:19:54,2014-10-04 11:19:54,2014-10-04 11:19:54,2014-10-04 11:20:00,2014-10-04 11:20:00,00:00:07
2,2014-07-03 11:00:28,2014-07-03 11:00:53,2014-07-03 11:00:58,2014-07-03 11:01:06,2014-07-03 11:01:09,2014-07-03 11:01:10,2014-07-03 11:01:23,2014-07-03 11:01:29,2014-07-03 11:01:30,2014-07-03 11:01:53,00:01:25
3,2014-12-05 15:55:12,2014-12-05 15:55:13,2014-12-05 15:55:14,2014-12-05 15:56:15,2014-12-05 15:56:16,2014-12-05 15:56:17,2014-12-05 15:56:18,2014-12-05 15:56:19,2014-12-05 15:56:33,2014-12-05 15:56:36,00:01:24


In [17]:
%%time
df_tr = new_df(train_df)
df_tr['session_timespan'] = time_train['delta'].dt.total_seconds().astype('int')
df_tr.head(3)

Wall time: 1min 51s


,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,day_of_week,start_hour,month,session_timespan
session_id,,,,,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0,4,10,2,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518,6,11,2,26
3,14769,39,14768,14769,37,39,14768,14768,14768,14768,1,16,12,7


In [18]:
%%time
df_test = new_df(test_df)
df_test['session_timespan'] = time_test['delta'].dt.total_seconds().astype('int')
df_test.head(3)

Wall time: 27.9 s


,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,day_of_week,start_hour,month,session_timespan
session_id,,,,,,,,,,,,,,
1,29,35,22,321,23,2211,6730,21,44582,15336,6,11,10,7
2,782,782,782,782,782,782,782,782,782,782,4,11,7,85
3,55,55,55,55,55,55,55,55,1445,1445,5,15,12,84


In [19]:
df_tr_test = pd.concat([df_tr, df_test])
df_tr_test.shape

(336358, 14)

**Создайте разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используйте объединенную матрицу *train_test_df_sites*, потом разделите обратно на обучающую и тестовую части.**

Обратите внимание на то, что в  сессиях меньше 10 сайтов  у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.

**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

In [20]:
def sparse(docs):
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}
    for d in docs:
        for term in d:
            if term == 0:
                break
            index = vocabulary.setdefault(term, len(vocabulary))
            indices.append(index)
            data.append(1)
        indptr.append(len(indices))

    X_sparse = csr_matrix((data, indices, indptr), dtype=int)
    return X_sparse

In [61]:
#для исходных данных 
train_test_sparse = sparse(train_test_df_sites.values)
test_size = len(test_df)/(len(train_df)+len(test_df))
X_train_sparse, X_test_sparse = train_test_split(train_test_sparse, test_size=test_size, shuffle=False)
y = train_df['target']

In [22]:
#для модифицированных данных
train_test_sparse1 = sparse(df_tr_test.values)
test_size1 = len(df_test)/(len(train_df)+len(df_test))
X_train_sp, X_test_sp = train_test_split(train_test_sparse1, test_size=test_size1, shuffle=False)
y = train_df['target']

**<font color='red'>Вопрос 1. </font> Выведите размерности матриц *X_train_sparse* и *X_test_sparse* – 4 числа на одной строке через пробел: число строк и столбцов матрицы *X_train_sparse*, затем число строк и столбцов матрицы *X_test_sparse*.**

In [23]:
print(X_train_sparse.shape[0], X_train_sparse.shape[1], X_test_sparse.shape[0], X_test_sparse.shape[1])

253561 48371 82797 48371


**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *kaggle_data/train_target.pkl*).**

In [26]:
with open(os.path.join(PATH_TO_DATA, 'X_train_sparse.pkl'), 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_test_sparse.pkl'), 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'train_target.pkl'), 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl, protocol=2)

**Разобьем обучающую выборку на 2 части в пропорции 7/3, причем не перемешивая. Исходные данные упорядочены по времени, тестовая выборка по времени четко отделена от обучающей, это же соблюдем и здесь.**

In [62]:
#для исходных данных 
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

In [25]:
#для модифицированных данных
train_share1 = int(.7 * X_train_sp.shape[0])
X_train1, y_train1 = X_train_sp[:train_share1, :], y[:train_share1]
X_valid1, y_valid1  = X_train_sp[train_share1:, :], y[train_share1:]

**Создайте объект `sklearn.linear_model.SGDClassifier` с логистической функцией потерь и параметром *random_state*=17. Остальные параметры оставьте по умолчанию, разве что *n_jobs*=-1 никогда не помешает. Обучите  модель на выборке `(X_train, y_train)`.**

In [63]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train, y_train)

SGDClassifier(loss='log', n_jobs=-1, random_state=17)

**Сделайте прогноз в виде предсказанных вероятностей того, что это сессия Элис, на отложенной выборке *(X_valid, y_valid)*.**

In [64]:
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)
logit_valid_pred_proba

array([[9.11911413e-01, 8.80885865e-02],
       [9.99703507e-01, 2.96493343e-04],
       [9.99872654e-01, 1.27345522e-04],
       ...,
       [9.87751880e-01, 1.22481202e-02],
       [9.99943934e-01, 5.60663172e-05],
       [9.88685254e-01, 1.13147460e-02]])

**<font color='red'>Вопрос 2. </font> Посчитайте ROC AUC логистической регрессии, обученной с помощью стохастического градиентного спуска, на отложенной выборке. Округлите до 3 знаков после разделителя.**

In [65]:
print(round(roc_auc_score(y_valid, logit_valid_pred_proba[:,1]),3))

0.934


**Сделайте прогноз в виде предсказанных вероятностей отнесения к классу 1 для тестовой выборки с помощью той же *sgd_logit*, обученной уже на всей обучающей выборке (а не на 70%).**

In [29]:
accuracy_score(y_valid, sgd_logit.predict(X_valid))

0.9909029959641904

In [30]:
%%time
logit = LogisticRegression(random_state=17,n_jobs=-1)
logit.fit(X_train1, y_train1)
accuracy_score(y_valid1, logit.predict(X_valid1))

Wall time: 2.75 s


0.9922307378827118

In [31]:
from sklearn.model_selection import RandomizedSearchCV

In [32]:
c_values1 = np.logspace(-2, 2,10)
c_values2 = np.linspace(0.1, 10, 30)

In [33]:
random_search = RandomizedSearchCV(estimator = logit, param_distributions={'C':c_values2}, scoring = 'roc_auc',
                           n_jobs=-1, cv=5, verbose=1)

In [34]:
%%time
random_search.fit(X_train1, y_train1)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Wall time: 28.2 s


RandomizedSearchCV(cv=5,
                   estimator=LogisticRegression(n_jobs=-1, random_state=17),
                   n_jobs=-1,
                   param_distributions={'C': array([ 0.1       ,  0.44137931,  0.78275862,  1.12413793,  1.46551724,
        1.80689655,  2.14827586,  2.48965517,  2.83103448,  3.17241379,
        3.5137931 ,  3.85517241,  4.19655172,  4.53793103,  4.87931034,
        5.22068966,  5.56206897,  5.90344828,  6.24482759,  6.5862069 ,
        6.92758621,  7.26896552,  7.61034483,  7.95172414,  8.29310345,
        8.63448276,  8.97586207,  9.31724138,  9.65862069, 10.        ])},
                   scoring='roc_auc', verbose=1)

In [35]:
random_search.best_score_, random_search.best_params_

(0.9720909194683454, {'C': 1.4655172413793105})

In [36]:
%%time
logit1 = LogisticRegression(random_state=17,n_jobs=-1, C=1.4655172413793105)
logit1.fit(X_train_sp, y)
logit_test_predict = logit1.predict_proba(X_test_sp)[:,1]
logit_test_predict2222 = logit1.predict(X_test_sp)

Wall time: 2.7 s


In [37]:
logit_test_predict

array([2.24464298e-04, 1.90004312e-06, 1.48745167e-07, ...,
       2.79095584e-03, 3.16356418e-05, 2.14577067e-07])

In [38]:
sum(logit_test_predict2222)

264

**Запишите ответы в файл и сделайте посылку на Kaggle. Дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera_Username", чтоб можно было легко идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard/public).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить.**

# Имя на leaderbord -  [YDF & MIPT] Kuzmin Sergey
# Score: 0.92353

In [28]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    print(predicted_df)
    predicted_df.to_csv(out_file, index_label=index_label)

In [48]:
write_to_submission_file(logit_test_predict, 
                         os.path.join(PATH_TO_DATA, 'logit_test_predict1.csv'))

## Критерии оценки работы (только для Peer Review в специализации):
- Правильные ли получились размерности матриц в п. 1? (max. 2 балла)
- Правильным ли получилось значения ROC AUC в п. 2? (max. 4 балла)
- Побит ли бенчмарк "sgd_logit_benchmark.csv" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)
- Побит ли бенчмарк "Logit +3 features" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)

## Пути улучшения
На этой неделе дается много времени на соревнование. Не забывайте вносить хорошие идеи, к которым Вы пришли по ходу соревнования, в описание финального проекта (`html`, `pdf` или `ipynb`). Это только в случае, если вы проходите специализацию.
Что можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели (проверить их можно на меньшей выборке по 150 пользователям, отделив одного из пользователей от остальных – это быстрее)
 - Настроить параметры моделей (например, коэффициенты регуляризации)
 - Если позволяют мощности (или хватает терпения), можно попробовать смешивание (блендинг) ответов бустинга и линейной модели. [Вот](http://mlwave.com/kaggle-ensembling-guide/) один из самых известных тьюториалов по смешиванию ответов алгоритмов, также хороша [статья](https://alexanderdyakonov.wordpress.com/2017/03/10/cтекинг-stacking-и-блендинг-blending) Александра Дьяконова
 - Обратите внимание, что в соревновании также даны исходные данные о посещенных веб-страницах Элис и остальными 1557 пользователями (*train.zip*). По этим данным можно сформировать свою обучающую выборку. 

На 6 неделе мы пройдем большой тьюториал по Vowpal Wabbit и попробуем его в деле, на данных соревнования.